# Creating the Distance Matrix

In order to calculate the access metrics, we need a distance matrix of each supply point to each demand point. In our case, from the census tract centroid to each school. 

Calculating the distance between every single pair of supply and demand points is computationally inviable, so we define a threshold for a maximum distance and we split the calculation into chunks. In practice, we divide our dataframe based on the microregions of Brazil and calculate the distance matrix of each one separately. The benefit is that we don't have to check the distance for two points that are hundreds of miles away. The downside is that with this method we ignore that people can cross microregions to go to school. However, I consider that this scenario is extremely uncommon. 

We will use the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula) to calculate the distance between two points on a sphere. The Haversine formula is a special case of the [Vincenty formula](https://en.wikipedia.org/wiki/Vincenty%27s_formulae), which is more accurate but also more computationally expensive. The Haversine formula is a good approximation for short distances, which is the case for our analysis.

W=

In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely import wkt
import warnings
from tqdm import tqdm

In [27]:

# importing basic dataset of the census tracts
geo_school_census_df = pd.read_csv('/Users/feliphlvo/Documents/Minerva/Capstone/data/Local/school_census.csv', index_col=0)
geo_dem_census_df = pd.read_csv('/Users/feliphlvo/Documents/Minerva/Capstone/data/Local/dem_census.csv', index_col=0)

In [28]:
# Only public schools with at least one high school class and regions with at least one high-school aged person
geo_dem_census_df = geo_dem_census_df[geo_dem_census_df["n_people_15to17"] > 0]
geo_school_census_df = geo_school_census_df[(geo_school_census_df["n_classes"] > 0) & (geo_school_census_df["admin_type"] != 4.0)]

In [29]:
# apply geometries
geo_dem_census_df = gpd.GeoDataFrame(geo_dem_census_df, geometry = geo_dem_census_df["geometry"].apply(wkt.loads))
geo_school_census_df = gpd.GeoDataFrame(geo_school_census_df, geometry = geo_school_census_df["geometry"].apply(wkt.loads))

In [33]:


warnings.filterwarnings('ignore')
def distance_matrix(demand_df, supply_df, demand_index, supply_index, name = "euclidean", threshold = 10000):   
    """""
    This function calculates the distance matrix between every pair of points in two geodataframes up to a threshold.

    Inspired by create_euclidean_distances in the Access package
    """

    # Reset the index so that the geoids are accessible
    df1 = demand_df.rename_axis("origin").reset_index()
    df2 = supply_df.rename_axis("dest").reset_index()
    
    # Convert to centroids if so-specified
    #print(df1.head())
    df1.set_geometry(df1.centroid, inplace=True)
    df2.set_geometry(df2.centroid, inplace=True)
    # Calculate the distances.
    if (df1.geom_type == "Point").all() & (df2.geom_type == "Point").all():
        # If both geometries are point types, merge on a temporary dummy column
        df1["temp"] = 1
        df2["temp"] = 1
        df1and2 = df1[["temp", "geometry", "origin"]].merge(
            df2[["temp", "geometry", "dest"]].rename(columns={"geometry": "geomb"})
        )
        df1and2.drop("temp", inplace=True, axis=1)
        df1and2[name] = df1and2.distance(df1and2.set_geometry("geomb"))
    else:
        # Execute an sjoin for non-point geometries, based upon a buffer zone
        df1and2 = gpd.sjoin(
            df1,
            df2.rename(columns={"geometry": "geomb"}).set_geometry(
                df2.buffer(threshold)
            ),
        )
        df1and2[name] = df1and2.distance(df1and2.set_geometry("geomb"))

    # Add it to the cost df.
    df1and2 = df1and2[df1and2[name] < threshold]
    df1and2 = df1and2.reset_index()[["origin", "dest", "euclidean"]]

    return df1and2


def chunked_distance_matrix(demand_df, supply_df, demand_index, supply_index, subset_column, name = "euclidean", threshold = 10000):
    '''''
    This function calculates the distance matrix between every pair of points in two geodataframes up to a threshold.
    It calculates distances separately for each subset to avoid memory issues.
    '''
    demand_df = demand_df.copy()
    supply_df = supply_df.copy()
    
    demand_df.set_index(demand_index, inplace=True)
    supply_df.set_index(supply_index, inplace=True)

    subset_values = demand_df[subset_column].unique()
    # Initialize distance matrix df
    distance_matrix_df = pd.DataFrame()
    for value in tqdm(subset_values):
        #print("Processing " + str(value))
        # Subset the dataframes
        demand_subset = demand_df[demand_df[subset_column] == value]
        supply_subset = supply_df[supply_df[subset_column] == value]
        # Calculate the distance matrix
        distance_matrix_df = distance_matrix_df.append(
            distance_matrix(demand_subset, supply_subset,  demand_index, supply_index, name, threshold)
        )
    return distance_matrix_df

In [34]:
type(geo_school_census_df)

geopandas.geodataframe.GeoDataFrame

In [35]:
dist_matrix = chunked_distance_matrix(geo_dem_census_df, geo_school_census_df, demand_index="sector_id",
                                      supply_index="school_id", subset_column="mesoregion_id", name="euclidean", threshold=16000)

100%|██████████| 138/138 [17:18<00:00,  7.53s/it]  


In [37]:
dist_matrix.describe()

,origin,dest,euclidean
count,2.045708e+07,2.045708e+07,2.045708e+07
mean,3.380449e+14,3.371970e+07,9.523882e+03
std,6.038906e+13,6.011901e+06,4.158382e+03
min,1.100015e+14,1.100026e+07,1.499205e+00
25%,3.303203e+14,3.305547e+07,6.276567e+03
50%,3.539806e+14,3.500491e+07,9.949617e+03
75%,3.550309e+14,3.540532e+07,1.309348e+04
max,5.300108e+14,5.306818e+07,1.600000e+04


In [38]:
# Write distance matrix to csv
dist_matrix.to_csv('/Users/feliphlvo/Documents/Minerva/Capstone/data/Local/dist_matrix.csv')